# University of Michigan
## School of Information
**Masters of Applied Data Science**<br>
**Milestone 1**<br>
<br>
Affordable Housing Project
<br><br>
**Submitted by**<br>
>Alan Fehsenfeld<br>Koon Leong Ho<br>George Thio

# Calculate Transit Scores

In [9]:
import pandas as pd
import geopandas
from config import basedir

base = basedir()

## Read Transit File

In [82]:
fields_to_retain=['stop_id','stop_lat','stop_lon','transit_authority']

data_dir = base + r"/data/raw"
fp_input_file = data_dir + r"/stops.txt"

# Grand_Rapids=pd.read_csv('documents/UofM/SIADS 591/GTFS Feeds/The Rapid/stops.txt')

Grand_Rapids=pd.read_csv(fp_input_file)

Grand_Rapids['transit_authority']=['The Rapid']*Grand_Rapids.shape[0]
Grand_Rapids=Grand_Rapids[fields_to_retain]
Grand_Rapids=geopandas.GeoDataFrame(Grand_Rapids,geometry=geopandas.points_from_xy(Grand_Rapids.loc[:,'stop_lon'],\
                                                                                  Grand_Rapids.loc[:,'stop_lat']))
Grand_Rapids=Grand_Rapids.set_crs('WGS84')
Grand_Rapids=Grand_Rapids.to_crs('EPSG:2253')
Grand_Rapids_geo=Grand_Rapids.buffer(1760)
Grand_Rapids.geometry=Grand_Rapids_geo

In [83]:
data_dir = base + r"/data/municipalities"
fp_input_file = data_dir + r"All Municipalities Geometry Only.shp"

Municipalities_file=geopandas.read_file(fp_input_file)

# Municipalities_file=geopandas.read_file('documents/UofM/SIADS 591/All Municipalities Geometry Only.shp')

## Merge Transit File and Municipalities File

In [231]:
transit=geopandas.sjoin(Grand_Rapids,Municipalities_file,how='right',op='intersects')
transit=transit.loc[:,['ID','transit_authority']]

In [232]:
transit['transit_authority']=transit['transit_authority'].where(transit['transit_authority'].isna(),5)
transit['transit_authority']=transit['transit_authority'].where(transit['transit_authority'].notna(),0)


In [233]:
transit=transit.groupby('ID').agg('max').reset_index()

In [234]:
transit=transit.rename(columns={'transit_authority':'transit_ind'})

In [235]:
data_dir = base + r"/data"
fp_output_file = data_dir + r"transit.csv"

transit.to_csv(fp_output_file,index=False)

# transit.to_csv('documents/UofM/SIADS 591/transit.csv',index=False)